In [2]:
import mysql.connector
import tkinter as tk
from datetime import datetime
from tkinter import ttk
import random


def buy_tickets(user_id):
    
    def submit_action():
        # Retrieve selected concert name from the dropdown
        selected_concert = concert_combobox.get()

        # Connect to MySQL database
        try:
            conn = mysql.connector.connect(
                host="localhost",
                user="enter your user_name here",
                password="enter your SQL DB password here",
                database="enter your DB name here"
            )
            cursor = conn.cursor()

            # Get the concert ID based on the selected concert name
            cursor.execute("SELECT concert_id FROM concert WHERE concert_name = %s", (selected_concert,))
            concert_id = cursor.fetchone()[0]

            # Generate a unique ticket ID
            cursor.execute("SELECT MAX(ticket_id) FROM tickets")
            max_ticket_id = cursor.fetchone()[0]
            new_ticket_id = max_ticket_id + 1 if max_ticket_id is not None else 1

            # Generate a random price between 50 and 80
            ticket_price = random.uniform(50, 80)

            # Insert into the ticket table
            cursor.execute("INSERT INTO tickets (ticket_id, price, user_id) VALUES (%s, %s, %s)",
                           (new_ticket_id, ticket_price, user_id))

            # Generate a unique seat number
            cursor.execute("SELECT MAX(seat_number) FROM seat")
            max_seat_number = cursor.fetchone()[0]
            new_seat_number = max_seat_number + 1 if max_seat_number is not None else 1

            # Insert into the seat table
            cursor.execute("INSERT INTO seat (seat_number, ticket_id, concert_id) VALUES (%s, %s, %s)",
                           (new_seat_number, new_ticket_id, concert_id))

            # Commit the changes
            conn.commit()

            # Display a success message
            success_label.config(text="Ticket booked successfully!")
            # Close the post window after successful submission
            ticket.destroy()
            

        except mysql.connector.Error as e:
            print("Error connecting to MySQL:", e)

        finally:
            if conn.is_connected():
                cursor.close()
                conn.close()

    # Function to open a new window for the booking page
    ticket = tk.Toplevel(root)
    ticket.title("Buy Tickets")
    ticket.geometry("750x750")
    
    # Intro title for the page
    main_label = tk.Label(ticket, text="Book your tickets here!!", font=('Arial', 16))
    main_label.pack(padx=20, pady=20)

    # Get all concert names from the database
    try:
        conn = mysql.connector.connect(
                host="localhost",
                user="enter your user_name here",
                password="enter your SQL DB password here",
                database="enter your DB name here"
        )
        cursor = conn.cursor()

        cursor.execute("SELECT concert_name FROM concert")
        concert_names = [row[0] for row in cursor.fetchall()]

    except mysql.connector.Error as e:
        print("Error connecting to MySQL:", e)
        concert_names = []

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

    # Dropdown menu for concert names
    concert_combobox = ttk.Combobox(ticket, values=concert_names, state="readonly", width=50)
    concert_combobox.pack()

    # Submit button for buying tickets
    submit_button = tk.Button(ticket, text="Buy Tickets", font=("Arial", 16), command=submit_action)
    submit_button.pack()

    # Success label for displaying successful ticket booking message
    success_label = tk.Label(ticket, text="", font=('Arial', 12), fg="green")
    success_label.pack()

    ticket.mainloop()



def get_info():
    
    def submit_action():
        # Retrieve selected concert name from the dropdown
        selected_concert = concert_combobox.get()

        # Connect to MySQL database
        try:
            conn = mysql.connector.connect(
                host="localhost",
                user="enter your user_name here",
                password="enter your SQL DB password here",
                database="enter your DB name here"
            )
            cursor = conn.cursor()

            # Query to get concert information (concert name, venue name, address)
            query = """
                SELECT c.concert_name, v.venue_name, a.street_number, a.street_name, a.city, a.zip_code
                FROM concert c
                JOIN venue v ON c.concert_id = v.concert_id
                JOIN address a ON v.venue_id = a.venue_id
                WHERE c.concert_name = %s
            """
            cursor.execute(query, (selected_concert,))
            result = cursor.fetchone()

            if result:
                # Display the concert information
                info_label.config(text=f"Concert: {result[0]}\nVenue: {result[1]}\nAddress: {result[2]}, {result[3]}, {result[4]}, {result[5]}")
            else:
                # Display an error message if the concert is not found
                info_label.config(text="Concert information not found")

        except mysql.connector.Error as e:
            print("Error connecting to MySQL:", e)

        finally:
            if conn.is_connected():
                cursor.close()
                conn.close()

    # Function to open a new window for the concert information page
    info = tk.Toplevel(root)
    info.title("Concert info")
    info.geometry("750x750")
    
    # Intro title for the page
    main_label = tk.Label(info, text="Concert Information", font=('Arial', 16))
    main_label.pack(padx=20, pady=20)
    
    # concert field
    concert_label = tk.Label(info, text="Select the name of the concert you're interested in:", font=('Arial', 12))
    concert_label.pack(anchor="w")
    
    # Get all concert names from the database
    try:
        conn = mysql.connector.connect(
                host="localhost",
                user="enter your user_name here",
                password="enter your SQL DB password here",
                database="enter your DB name here"
        )
        cursor = conn.cursor()

        cursor.execute("SELECT concert_name FROM concert")
        concert_names = [row[0] for row in cursor.fetchall()]

    except mysql.connector.Error as e:
        print("Error connecting to MySQL:", e)
        concert_names = []

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

    # Dropdown menu for concert names
    concert_combobox = ttk.Combobox(info, values=concert_names, state="readonly", width=50)
    concert_combobox.pack()

    # Submit button for concert information
    submit_button = tk.Button(info, text="Submit", font=("Arial", 16), command=submit_action)
    submit_button.pack()

    # Label for displaying concert information
    info_label = tk.Label(info, text="", font=('Arial', 12))
    info_label.pack()

    info.mainloop()

    


def delete_page(user_id):
    
    def delete_action():
        # Retrieve comment to delete from the Text widget
        review_to_delete = review_box.get("1.0", "end-1c")  # Get text from Text widget

        # Connect to MySQL database
        try:
            conn = mysql.connector.connect(
                host="localhost",
                user="enter your user_name here",
                password="enter your SQL DB password here",
                database="enter your DB name here"
            )
            cursor = conn.cursor()

            # Query the review table to check if the review exists
            query = "SELECT * FROM review WHERE comment = %s AND user_id = %s"
            cursor.execute(query, (review_to_delete, user_id))
            result = cursor.fetchone()

            if result:
                # if review exists, proceed with deletion
                delete_query = "DELETE FROM review WHERE comment = %s AND user_id = %s"
                cursor.execute(delete_query, (review_to_delete, user_id))
                conn.commit()
                
                # Close the delete_review window after successful deletion
                delete_review.destroy()
            else:
                # Display an error message if the review does not exist
                error_label.config(text="No such review exists")

        except mysql.connector.Error as e:
            print("Error connecting to MySQL:", e)

        finally:
            if conn.is_connected():
                cursor.close()
                conn.close()
    
    # Function to open a new window for the delete_review page
    delete_review = tk.Toplevel(root)
    delete_review.title("Delete reviews")
    delete_review.geometry("750x750")
    
    # Intro title for the page
    main_label = tk.Label(delete_review, text="Delete your reviews here", font=('Arial', 16))
    main_label.pack(padx=20, pady=20)
    
    # delete post field
    del_label = tk.Label(delete_review, text="Enter the review that you want to delete", font=('Arial', 12))
    del_label.pack(anchor="w")
    # Text box
    review_box = tk.Text(delete_review, height=5, width=100)
    review_box.pack()
    
    # Submit button for review deletion
    submit_button = tk.Button(delete_review, text="Submit", font=("Arial", 16), command=delete_action)
    submit_button.pack()
    
    # error msg
    error_label = tk.Label(delete_review, text="", font=('Arial', 12), fg="red")
    error_label.pack()
    
    delete_review.mainloop()
 
    
    
def post_reviews_page(user_id):
    
    def submit_action():
        # Retrieve rating and review from the widgets
        entered_rating = rating.get()
        entered_comment = post_box.get("1.0", "end-1c")  # Get text from Text widget
        entered_date_posted = datetime.now().strftime("%Y-%m-%d")  # Get current date in 'yyyy-mm-dd' format

        # Connect to MySQL database
        try:
            conn = mysql.connector.connect(
                host="localhost",
                user="enter your user_name here",
                password="enter your SQL DB password here",
                database="enter your DB name here"
            )
            cursor = conn.cursor()

            # Insert into review table
            query = "INSERT INTO review (rating, comment, date_posted, user_id) VALUES (%s, %s, %s, %s)"
            cursor.execute(query, (entered_rating, entered_comment, entered_date_posted, user_id))

            # Commit the changes
            conn.commit()

            # Close the post window after successful submission
            post.destroy()

        except mysql.connector.Error as e:
            print("Error connecting to MySQL:", e)

        finally:
            if conn.is_connected():
                cursor.close()
                conn.close()
                
    # Function to open a new window for the review page
    post = tk.Toplevel(root)
    post.title("Post reviews")
    post.geometry("750x750")
    
    # Intro title for the page
    main_label = tk.Label(post, text="Thanks for giving us your feedback", font=('Arial', 16))
    main_label.pack(padx=20, pady=20)
    
    
    # rating field
    rating_label = tk.Label(post, text="Enter rating (From 1 through 10)", font=('Arial', 12))
    rating_label.pack(anchor="w")
    # variable to store name
    rating = tk.StringVar()
    # Text box
    rating_box = tk.Entry(post, textvariable= rating, width=50)
    rating_box.pack()
    # blank line
    blank_line = tk.Label(post, text=" ", height=2)
    blank_line.pack()
    
    
    # post field
    post_label = tk.Label(post, text="Please enter your review here and hit submit!!", font=('Arial', 12))
    post_label.pack(anchor="w")
    # variable to store name
    post_var = tk.StringVar()
    # Text box
    post_box = tk.Text(post, height=5, width=100)
    post_box.pack()
    
    submit_button = tk.Button(post, text="Submit", font=("Arial", 16), command=submit_action)
    submit_button.pack()
    
    post.mainloop()

    
def open_main_page(user_id):
    # Function to open a new window for the sign-up page
    main_page = tk.Toplevel(root)
    main_page.title("Main Page")
    main_page.geometry("750x750")

    # Intro title for the page
    main_label = tk.Label(main_page, text="Welcome to the Main Page!", font=('Arial', 16))
    main_label.pack(padx=20, pady=20)
    
    # Book your ticket button 
    ticket_button = tk.Button(main_page, text="Book tickets", font=("Arial", 16), width=25, height=1, command=lambda: buy_tickets(user_id))
    ticket_button.pack()
    # blank line
    blank_line = tk.Label(main_page, text=" ", height=2)
    blank_line.pack()
    
    # concert info button 
    info_button = tk.Button(main_page, text="View concert information", font=("Arial", 16), width=25, height=1, command = get_info)
    info_button.pack()
    # blank line
    blank_line = tk.Label(main_page, text=" ", height=2)
    blank_line.pack()
    
    # post review button
    review_post_button = tk.Button(main_page, text="Post your reviews", font=("Arial", 16), width=25, height=1, command=lambda: post_reviews_page(user_id))
    review_post_button.pack()
    # blank line
    blank_line = tk.Label(main_page, text=" ", height=2)
    blank_line.pack()
    
    # delete review button
    delete_review_button = tk.Button(main_page, text="Delete your reviews", font=("Arial", 16), width=25, height=1, command=lambda: delete_page(user_id))
    delete_review_button.pack()
    # blank line
    blank_line = tk.Label(main_page, text=" ", height=2)
    blank_line.pack()
       
    main_page.mainloop()
    
   
def sign_up_page():
    # Function to handle the sign-up submission
    def submit_action():
        # Retrieve user details from the Entry widgets
        entered_name = name.get()
        entered_password = password.get()
        entered_email = email.get()
        entered_phone = phone.get()

        # Connect to MySQL database
        try:
            conn = mysql.connector.connect(
                host="localhost",
                user="enter your user_name here",
                password="enter your SQL DB password here",
                database="enter your DB name here"
            )
            cursor = conn.cursor()
            
            # Get the maximum user_id from the app_user table
            cursor.execute("SELECT MAX(user_id) FROM app_user")
            max_user_id = cursor.fetchone()[0]

            # Increment the maximum user_id by 1 or set to 1 if no users exist
            user_id = max_user_id + 1 if max_user_id else 1

            # Insert into app_user table
            query = "INSERT INTO app_user (user_id, password, name, email, phone) VALUES (%s, %s, %s, %s, %s)"
            cursor.execute(query, (user_id, entered_password, entered_name, entered_email, entered_phone))

            # Commit the changes
            conn.commit()

            # Display a success message
            success_label.config(text="Signup successful! UserId will be sent to your Email")
            # Close the signup window after successful submission
            sign_up.destroy()

        except mysql.connector.Error as e:
            print("Error connecting to MySQL:", e)

        finally:
            if conn.is_connected():
                cursor.close()
                conn.close()

    # Function to open a new window for the sign-up page
    sign_up = tk.Toplevel(root)
    sign_up.title("Signup Page")
    sign_up.geometry("750x750")

    # Intro title for the page
    title_label = tk.Label(sign_up, text="Welcome to the SignUp Page!", font=('Arial', 16))
    title_label.pack(padx=20, pady=20)

    # name field
    name_label = tk.Label(sign_up, text="Please enter your name:", font=('Arial', 12))
    name_label.pack(anchor="w")
    # variable to store name
    name = tk.StringVar()
    # Text box
    name_box = tk.Entry(sign_up, textvariable=name, width=50)
    name_box.pack()
    # blank line
    blank_line = tk.Label(sign_up, text=" ", height=2)
    blank_line.pack()

    # password field
    pass_label = tk.Label(sign_up, text="Enter password:", font=('Arial', 12))
    pass_label.pack(anchor="w")
    # variable to store password
    password = tk.StringVar()
    # Text box
    pass_box = tk.Entry(sign_up, textvariable=password, width=50)
    pass_box.pack()
    # blank line
    blank_line = tk.Label(sign_up, text=" ", height=2)
    blank_line.pack()

    # email
    email_label = tk.Label(sign_up, text="Your Email Id:", font=('Arial', 12))
    email_label.pack(anchor="w")
    # variable to store email
    email = tk.StringVar()
    # Text box
    email_box = tk.Entry(sign_up, textvariable=email, width=50)
    email_box.pack()
    # blank line
    blank_line = tk.Label(sign_up, text=" ", height=2)
    blank_line.pack()

    # phone
    phone_label = tk.Label(sign_up, text="Your phone number:", font=('Arial', 12))
    phone_label.pack(anchor="w")
    # variable to store phone number
    phone = tk.StringVar()
    # Text box
    phone_box = tk.Entry(sign_up, textvariable=phone, width=50)
    phone_box.pack()

    # Submit button
    submit_button = tk.Button(sign_up, text="Submit", font=("Arial", 16), command=submit_action)
    submit_button.pack()

    # Success label for displaying successful signup message
    success_label = tk.Label(sign_up, text="", font=('Arial', 12), fg="green")
    success_label.pack()

    sign_up.mainloop()

    

def open_login_page():
    # Function to handle the submit action
    def submit_action():
        # Retrieve user_id and password from the Entry widgets
        entered_user_id = user_id_var.get()
        entered_password = password_var.get()

        # Connect to MySQL database
        try:
            conn = mysql.connector.connect(
                host="localhost",
                user="enter your user_name here",
                password="enter your SQL DB password here",
                database="enter your DB name here"
            )
            cursor = conn.cursor()

            # Query the app_user table
            query = "SELECT * FROM app_user WHERE user_id = %s AND password = %s"
            cursor.execute(query, (entered_user_id, entered_password))
            result = cursor.fetchone()

            if result:
                # Successful login
                login_page.destroy() 
                open_main_page(entered_user_id)
            else:
                # Invalid login
                error_label.config(text="Invalid user ID or password")

        except mysql.connector.Error as e:
            print("Error connecting to MySQL:", e)

        finally:
            if conn.is_connected():
                cursor.close()
                conn.close()

    # Function to open a new window for the login page
    login_page = tk.Toplevel(root)
    login_page.title("Login Page")
    login_page.geometry("750x750")

    # Intro title for the page
    title_label = tk.Label(login_page, text="Welcome to the Login Page!", font=('Arial', 16))
    title_label.pack(padx=20, pady=20)

    # user_id field
    user_id_label = tk.Label(login_page, text="Enter your user ID:", font=('Arial', 12))
    user_id_label.pack(anchor="w")

    # variable to store user_id
    user_id_var = tk.IntVar()
    # Text box
    user_id_box = tk.Entry(login_page, textvariable=user_id_var, width=50)
    user_id_box.pack()

    # blank line
    blank_line = tk.Label(login_page, text=" ", height=2)
    blank_line.pack()

    # password field
    password_label = tk.Label(login_page, text="Enter your password:", font=('Arial', 12))
    password_label.pack(anchor="w")

    # variable to store password
    password_var = tk.StringVar()
    # Text box
    password_box = tk.Entry(login_page, textvariable=password_var, width=50)
    password_box.pack()

    # Submit button
    submit_button = tk.Button(login_page, text="Submit", font=("Arial", 16), command=submit_action)
    submit_button.pack()

    # Error label for displaying invalid login message
    error_label = tk.Label(login_page, text="", font=('Arial', 12), fg="red")
    error_label.pack()

    login_page.mainloop()


# Main function
root = tk.Tk()
root.geometry("750x750")
root.title("Concert reservation system")

# Intro title for the page
label = tk.Label(root, text="Welcome to Boston Concert Reservation System", font=('Arial', 18))
label.pack(padx=20, pady=20)

intro_sentence = '''Experience the magic of live music on our platform – your hub for concert reviews, ticket 
availability checks, and personalized ticket insights. Join a vibrant community, discover upcoming events, 
and effortlessly secure your spot at the hottest concerts. Your gateway to a dynamic concert experience  
begins here!!
'''
intro = tk.Label(root, text=intro_sentence, font=('Arial', 12), anchor="e", justify="left")
intro.pack()

# login button
login_button = tk.Button(root, text="Login", font=("Arial", 16), width=17, height=1, command=open_login_page)
login_button.pack()
empty_line = tk.Label(root, text=" ", height=1)
empty_line.pack()

# signup button
sign_up = tk.Button(root, text="Sign up", font=("Arial", 16), width=17, height=1, command= sign_up_page)
sign_up.pack()

root.mainloop()
